# Sales Forecasting using XGBoost

In [764]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [765]:
df = pd.read_csv('supermarket_sales - Sheet1.csv')

In [766]:
df.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [767]:
Q1 = df["gross income"].quantile(0.25)
Q3 = df["gross income"].quantile(0.75)

IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
filtered_data = df['gross income'][(df["gross income"] >= lower_bound) & (df["gross income"]<= upper_bound)]
df['Filtered_data']=filtered_data

In [768]:
df.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating,Filtered_data
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1,26.1415
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6,3.8200
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4,16.2155
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4,23.2880
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3,30.2085


In [769]:
df.isnull().sum()

Invoice ID                 0
Branch                     0
City                       0
Customer type              0
Gender                     0
Product line               0
Unit price                 0
Quantity                   0
Tax 5%                     0
Total                      0
Date                       0
Time                       0
Payment                    0
cogs                       0
gross margin percentage    0
gross income               0
Rating                     0
Filtered_data              9
dtype: int64

In [770]:
df["gross income"].mean()

15.379368999999999

In [771]:
df["Filtered_data"].fillna(15.379368999999999, inplace=True)

In [772]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df["New_City"]=le.fit_transform(df["City"])
df.drop("City",axis=1,inplace=True)

df["Customer_type"]=le.fit_transform(df["Customer type"])
df.drop("Customer type",axis=1,inplace=True)

df["New_branch"]=le.fit_transform(df["Branch"])
df.drop("Branch",axis=1,inplace=True)


df["New_Gender"]=le.fit_transform(df["Gender"])
df.drop("Gender",axis=1,inplace=True)
                                
    
df["New_Product_line "]=le.fit_transform(df["Product line"])
df.drop("Product line",axis=1,inplace=True)
                                     
df["New_Payment"]=le.fit_transform(df["Payment"])
df.drop("Payment",axis=1,inplace=True)
                                     

In [773]:
df

,Invoice ID,Unit price,Quantity,Tax 5%,Total,Date,Time,cogs,gross margin percentage,gross income,Rating,Filtered_data,New_City,Customer_type,New_branch,New_Gender,New_Product_line,New_Payment
0,750-67-8428,74.69,7,26.1415,548.9715,1/5/2019,13:08,522.83,4.761905,26.1415,9.1,26.141500,2,0,0,0,3,2
1,226-31-3081,15.28,5,3.8200,80.2200,3/8/2019,10:29,76.40,4.761905,3.8200,9.6,3.820000,1,1,2,0,0,0
2,631-41-3108,46.33,7,16.2155,340.5255,3/3/2019,13:23,324.31,4.761905,16.2155,7.4,16.215500,2,1,0,1,4,1
3,123-19-1176,58.22,8,23.2880,489.0480,1/27/2019,20:33,465.76,4.761905,23.2880,8.4,23.288000,2,0,0,1,3,2
4,373-73-7910,86.31,7,30.2085,634.3785,2/8/2019,10:37,604.17,4.761905,30.2085,5.3,30.208500,2,1,0,1,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,233-67-5758,40.35,1,2.0175,42.3675,1/29/2019,13:46,40.35,4.761905,2.0175,6.2,2.017500,1,1,2,1,3,2
996,303-96-2227,97.38,10,48.6900,1022.4900,3/2/2019,17:16,973.80,4.761905,48.6900,4.4,15.379369,0,1,1,0,4,2
997,727-02-1313,31.84,1,1.5920,33.4320,2/9/2019,13:22,31.84,4.761905,1.5920,7.7,1.592000,2,0,0,1,2,0
998,347-56-2442,65.82,1,3.2910,69.1110,2/22/2019,15:33,65.82,4.761905,3.2910,4.1,3.291000,2,1,0,1,4,0


In [774]:
X=df.drop(['Filtered_data','Time','Invoice ID'],axis=1)
y=df["Filtered_data"]
y.index = X['Date']
X.drop('Date',axis = 1,inplace = True)
y

Date
1/5/2019     26.141500
3/8/2019      3.820000
3/3/2019     16.215500
1/27/2019    23.288000
2/8/2019     30.208500
               ...    
1/29/2019     2.017500
3/2/2019     15.379369
2/9/2019      1.592000
2/22/2019     3.291000
2/18/2019    30.919000
Name: Filtered_data, Length: 1000, dtype: float64

In [775]:
X

,Unit price,Quantity,Tax 5%,Total,cogs,gross margin percentage,gross income,Rating,New_City,Customer_type,New_branch,New_Gender,New_Product_line,New_Payment
0,74.69,7,26.1415,548.9715,522.83,4.761905,26.1415,9.1,2,0,0,0,3,2
1,15.28,5,3.8200,80.2200,76.40,4.761905,3.8200,9.6,1,1,2,0,0,0
2,46.33,7,16.2155,340.5255,324.31,4.761905,16.2155,7.4,2,1,0,1,4,1
3,58.22,8,23.2880,489.0480,465.76,4.761905,23.2880,8.4,2,0,0,1,3,2
4,86.31,7,30.2085,634.3785,604.17,4.761905,30.2085,5.3,2,1,0,1,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,40.35,1,2.0175,42.3675,40.35,4.761905,2.0175,6.2,1,1,2,1,3,2
996,97.38,10,48.6900,1022.4900,973.80,4.761905,48.6900,4.4,0,1,1,0,4,2
997,31.84,1,1.5920,33.4320,31.84,4.761905,1.5920,7.7,2,0,0,1,2,0
998,65.82,1,3.2910,69.1110,65.82,4.761905,3.2910,4.1,2,1,0,1,4,0


In [776]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(X,y, test_size=0.2, random_state=42)

In [782]:
rr=Ridge(alpha=0.0001)
rr.fit(x_train, y_train)

Ridge(alpha=0.0001)

In [784]:
y_pred = rr.predict(x_test)
np.mean(cross_val_score(rr,X,y,cv=10, scoring='r2'))

0.922121705088131

In [682]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import FunctionTransformer

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

In [683]:
data = df.drop(['Invoice ID','Tax 5%','Date','Time','cogs','gross margin percentage','City','Total','Unit price','Quantity'], axis=1)

In [684]:
data.head()

,Branch,Customer type,Gender,Product line,Payment,gross income,Rating
0,A,Member,Female,Health and beauty,Ewallet,26.1415,9.1
1,C,Normal,Female,Electronic accessories,Cash,3.8200,9.6
2,A,Normal,Male,Home and lifestyle,Credit card,16.2155,7.4
3,A,Member,Male,Health and beauty,Ewallet,23.2880,8.4
4,A,Normal,Male,Sports and travel,Ewallet,30.2085,5.3


In [685]:
X=data.drop('Rating', axis=1)
y=data['Rating']

In [686]:
X

,Branch,Customer type,Gender,Product line,Payment,gross income
0,A,Member,Female,Health and beauty,Ewallet,26.1415
1,C,Normal,Female,Electronic accessories,Cash,3.8200
2,A,Normal,Male,Home and lifestyle,Credit card,16.2155
3,A,Member,Male,Health and beauty,Ewallet,23.2880
4,A,Normal,Male,Sports and travel,Ewallet,30.2085
...,...,...,...,...,...,...
995,C,Normal,Male,Health and beauty,Ewallet,2.0175
996,B,Normal,Female,Home and lifestyle,Ewallet,48.6900
997,A,Member,Male,Food and beverages,Cash,1.5920
998,A,Normal,Male,Home and lifestyle,Cash,3.2910


In [687]:
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [714]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import Ridge, Lasso

In [715]:
#ohe the categorical features
trf1 = ColumnTransformer(transformers = [
    ('ohe', OneHotEncoder(drop='first', sparse_output=False), [0,1,2,3,4])
], remainder='passthrough')

trf3 = ColumnTransformer(transformers=[
    ('SS', StandardScaler(), [5]),
    #('minmax', MinMaxScaler(), [4])
], remainder='passthrough')

In [716]:
trf4 = Lasso(alpha=0.001)
trf5 = RandomForestRegressor()

In [717]:
pipe1 = Pipeline([
    ('trf1', trf1),
    ('trf3', trf3),
    ('trf4', trf4)
])

In [718]:
pipe1.fit(x_train, y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  [0, 1, 2, 3, 4])])),
                ('trf3',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('SS', StandardScaler(),
                                                  [5])])),
                ('trf4', Lasso(alpha=0.001))])

In [719]:
y_pred1 = pipe1.predict(x_test)

In [720]:
r2_score(y_test, y_pred1)

-0.016031035393647475

In [785]:
colors = ['#F0D290','#DE834D','#A3423C','#781D42']
def plot_forecasted_sales(y_pred, y_train, y_true=[], period=5, title=''):
    
    # plot
    plt.figure(figsize=(20,12))
    plt.plot(y_train.to_list(), color=colors[0])
    plt.plot([None for i in y_train] + [x for x in y_pred], color=colors[2])
    
    
    # set legend
    if len(y_true) > 0:
        plt.plot([None for i in y_train] + [x for x in y_true], color=colors[1])
        plt.legend(['2014-2016 Acual Income', '2017 Acual Income', f'2017 Forecasted Income \
(RMSE = {mean_squared_error(y_true, y_pred, squared=False).round(1)}, r2 = {r2_score(y_true, y_pred).round(3)})'])
    else:
        plt.legend(['2014-2017 Actual Income', '2018-2020 Forecasted Income'])
    
    # set parameters
    plt.xticks(range(0, len(y_train) + len(y_pred), period), rotation=75)
    plt.xlabel('Week')
    plt.ylabel('Sales')
    plt.title(title)
    plt.show()
    
plot_forecasted_sales(pred,y_train)

NameError: name 'pred' is not defined